In [1]:
from langchain_utils import load_data, get_questions, generate_search_queries_prompt
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
import os
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
# display as markdown
from IPython.display import Markdown, display
import openai


In [2]:
query = "which covid 19 vaccine is most effective?"
def get_related_questions(query):
    research_template = """You are a search engine expert"""
            
    openai.api_key = os.getenv("OPENAI_API_KEY")

    messages = [{
                "role": "system",
                "content": research_template
            }, {
                "role": "user",
                "content": generate_search_queries_prompt(query),
            }]

    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0.5,
    max_tokens=256
    )
    related_questions = get_questions(response.choices[0].message.content)
    related_questions.append(query)
    return related_questions

related_questions = get_related_questions(query)


In [3]:
related_questions

['efficacy comparison of covid 19 vaccines',
 'which covid 19 vaccine is most effective?']

In [4]:
docs = []

In [5]:
for ques in related_questions:
    _docs = load_data(ques, 50)
    docs.extend(_docs)

In [6]:
docs

[Document(page_content='Efficacy and safety of COVID‐19 vaccines: A network meta‐analysis;\n ABSTRACT:Several vaccines showed a good safety profile and significant efficacy against COVID‐19. Moreover, in the absence of direct head to head comparison between COVID‐19 vaccines, a network meta‐analysis that indirectly compares between them is needed.', metadata={'title': 'Efficacy and safety of COVID‐19 vaccines: A network meta‐analysis', 'venue': 'Journal of Evidence-Based Medicine', 'year': 2022, 'paperId': '98a08ff1cc78ff55e1c47d67d02a3b4431cedc32', 'citationCount': 6, 'openAccessPdf': None, 'authors': ['A. A. Toubasi', 'Thuraya Al-Sayegh', 'Yazan Y. Obaid', 'Sarah M Al-Harasis', 'S. A. Alryalat'], 'externalIds': {'PubMedCentral': '9538745', 'DOI': '10.1111/jebm.12492', 'CorpusId': 251765800, 'PubMed': '36000160'}}),
 Document(page_content='Efficacy of COVID-19 vaccines: a systematic review and network meta-analysis of phase 3 randomized controlled trials', metadata={'title': 'Efficacy

In [7]:
def get_unique_docs(docs):
    unique_docs_id = []
    unique_docs = []
    for doc in docs:
        if doc.metadata['paperId'] not in unique_docs:
            unique_docs_id.append(doc.metadata['paperId'])
            unique_docs.append(doc)
    return unique_docs

In [8]:
len(get_unique_docs(docs))

100

In [9]:

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [10]:
docs = db.similarity_search(query)
print(docs[0].page_content)

Which COVID-19 Vaccine is Efficiently Most Cost-Effective and Why?;
 ABSTRACT:This study presents the dynamic boost of the COVID-19 vaccines. The intensive approach is indicating how the 
 effective COVID-19 vaccine promotes and distributes into various official channels. The Cost-Effectiveness Analysis 
 (CEA) approach is used with Quality-Adjusted Life Years (QALYs). The key findings are revealing the length of life 
 improves the quality of life of the middle family. The QALYs show that Utility Value (UV) by health and the value of 
 statistics in countries in the term of coexistent causation of vaccines. The Pfizer (BNT162b2) and Moderna (mRNA- 
 1273) have instantly created favourable and significant effects probably on many patients, comparatively SARSCoV- 
 2, AstraZeneca (AZD1222), Russia’s Sputnik (AstraZeneca), and Sinopharm Sinovac Biotech. The opportunity 
cost provides a valuable benefit in the future.


In [11]:
# from langchain.chains.question_answering import load_qa_chain
# from langchain.llms import OpenAI

chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")

In [12]:
response = chain.run(input_documents=docs, question=query)
display(Markdown(response))

 The most effective COVID-19 vaccine appears to be the BNT162b2 vaccine, with an efficacy rate of 95%.